In [1]:
import numpy as np
import cmath
import pyperclip
import geogebra
import limb
import movement_assignment
import checks

In [2]:
class platform:
    def __init__(self,d,D,l1,l2,max_limb_error,max_limb_try_count,max_joint_angle):
        "d: distancia de cada junta universal en la plataforma al centro"
        
        "D: distancia del eje de cada motor al centro de la base"
        
        "la altura del eje del motor es conciderada el plano z=0"
        
        "l1: long del antebrazo"
        
        "l2: long del brazo"
        
        "max_limb_error: máximo error (en las mismas unidades de l1 y l2) que es permitido para posicionar el brazo"
        
        "max_limb_try_count: máxima cantidad de iteraciones para calcular el ángulo de un motor, si la cantidad"
        "Si la cantidad requerida de iteraciones es mayor, genera un error llamado 'Point unreachable' "
        
        "max_joint_angle:angulo máximo que puede tener la junta universal, una posición que lo exceda genera un error"
        "llamado 'Joint angle exceeded'"
        
        self.c1=np.array([0,-d,0])
        self.c2=np.array([-d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        self.c3=np.array([d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        
        self.d=d
        self.D=D

        self.l1=l1
        self.l2=l2
        
        self.max_limb_error=max_limb_error
        self.max_limb_try_count=max_limb_try_count
        
        #defino los angulos de los motores
        self.ang1=0
        self.ang2=0
        self.ang3=0
        #defino los angulos máximos que se les permite rotar a los motores
        self.angle_max=90
        self.angle_min=-90
        #defino el ángulo máximo permitido en las juntas universales
        self.max_joint_angle=max_joint_angle
    def x_rotation_matrix(self,r_x):
        return np.array([[1,0,0],[0,np.cos(r_x),-np.sin(r_x)],[0,np.sin(r_x),np.cos(r_x)]]) 
    
    def y_rotation_matrix(self,r_y):
        return np.array([[np.cos(r_y),0,np.sin(r_y)],[0,1,0],[-np.sin(r_y),0,np.cos(r_y)]])
          
    def rotate(self,movement,joint,angle):
        if movement=="pitch":
            M_r=self.x_rotation_matrix(angle)
        if movement=="roll":
            M_r=self.y_rotation_matrix(angle)
            
        if joint=="C1":
            wm=np.identity(3)
        if joint=="C2":
            wm=self.get_m2_basis()
        if joint=="C3":
            wm=self.get_m3_basis()
        #transformamos los puntos a la base conveniente
        self.c1=np.linalg.solve(wm,self.c1)
        self.c2=np.linalg.solve(wm,self.c2)
        self.c3=np.linalg.solve(wm,self.c3)
        #realizamos la rotación de los mismos
        self.c1=M_r.dot(self.c1)
        self.c2=M_r.dot(self.c2)
        self.c3=M_r.dot(self.c3)
        #corregimos el offset de la plataforma
        if joint=="C1":
            self.apply_offset(self.c2)
        if joint=="C2":
            self.apply_offset(self.c3)
        if joint=="C3":
            self.apply_offset(self.c1)
        #obtengo los puntos en la base original
        self.c1=wm.dot(self.c1)
        self.c2=wm.dot(self.c2)  
        self.c3=wm.dot(self.c3)
        
    def apply_offset (self,pos):
        "la función desplaza la plataforma el offset correspondiente para compenzar el introducido por la rotación"
        allowed_pos=-pos[0]*np.tan(np.pi/6)
        offset=np.array([0,allowed_pos-pos[1],0])
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        
    def get_m2_basis(self):
        Wm2=np.array([[-np.cos(np.pi/3),np.cos(np.pi/6),0],[-np.sin(np.pi/3),-np.sin(np.pi/6),0],[0,0,1]])
        return Wm2

    def get_m3_basis(self):
        Wm3=np.array([[-np.cos(np.pi/3),-np.cos(np.pi/6),0],[np.sin(np.pi/3),-np.sin(np.pi/6),0],[0,0,1]])
        return Wm3
    
        
    def calculate_P(self):
            wm2=self.get_m2_basis()
            wm3=self.get_m3_basis()
            P1=self.c1+np.array([0,self.D,0])
            P2=np.linalg.solve(wm2,self.c2)+np.array([0,self.D,0])
            P3=np.linalg.solve(wm3,self.c3)+np.array([0,self.D,0])
            #elimino la coordenana en x
            P1=np.array([P1[1],P1[2]])
            P2=np.array([P2[1],P2[2]])
            P3=np.array([P3[1],P3[2]])
            return P1,P2,P3
    

        
    def correct_height(self,h):
        c1x=self.c1[0]
        c2x=self.c2[0]
        c3x=self.c3[0]
        
        c1y=self.c1[1]
        c2y=self.c2[1]
        c3y=self.c3[1]
        
        c1z=self.c1[2]
        c2z=self.c2[2]
        c3z=self.c3[2]
        
        A=np.array([[c1x,c1y,1],[c2x,c2y,1],[c3x,c3y,1]])
        B=np.array([c1z,c2z,c3z]).transpose()
        params=np.linalg.solve(A,B)
        offset=np.array([0,0,h-params[2]])
        #corrijo la altura
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        

    def solve_platform(self,r_x,r_y,h):
        if r_x==0 and r_y==0:
            #Este if se utiliza para no calcular rotaciones cuando los ángulos de ambos ejes son cero
            self.correct_height(h)
        else:
            joint,movement,angle=movement_assignment.assign_movement(r_x,r_y)
            self.rotate(movement,joint,angle)
            self.correct_height(h)
        
        P1,P2,P3=self.calculate_P()
        limb_i=limb.limb(self.l1,self.l2)
        self.ang1=limb_i.calculate_motor_angle(P1,self.max_limb_error,self.max_limb_try_count)
        self.ang2=limb_i.calculate_motor_angle(P2,self.max_limb_error,self.max_limb_try_count)
        self.ang3=limb_i.calculate_motor_angle(P3,self.max_limb_error,self.max_limb_try_count)
        #Chekeo el ángulo necesario en las juntas universales
        for joint in ["C1","C2","C3"]:
            angle=checks.get_joint_angle(self.c1,self.c2,self.c3,joint)
            if np.abs(angle)> self.max_joint_angle:
                #raise ValueError('Joint angle exceeded\n {}.angle={}º >{}º'.format(joint,angle*180/np.pi,self.max_joint_angle*180/np.pi))
                print('Joint angle exceeded\n {}.angle={}º >{}º'.format(joint,angle*180/np.pi,self.max_joint_angle*180/np.pi))
                return -1
            else:
                return 0       

In [3]:
angle_spacex=np.linspace(start=-19*np.pi/180,stop=19*np.pi/180,num=10)
angle_spacey=np.linspace(start=-19*np.pi/180,stop=19*np.pi/180,num=10)
h_space=np.linspace(start=7,stop=7,num=1)
s1=geogebra.geogebra_script()
point=0
for height in [10,11,12,13,14]:
    for angle_x in angle_spacex:
        for angle_y in angle_spacey:
            p1=platform(d=6,D=10,l1=6,l2=12,max_limb_error=0.00001,max_limb_try_count=30,max_joint_angle=23*np.pi/180)
            result=p1.solve_platform(r_x=angle_x,r_y=angle_y,h=height) 
            if result==-1:
                print("angle exceeded")
                print("angle_x={}".format(angle_x*180/np.pi))
                print("angle_y={}".format(angle_y*180/np.pi))
            else:
                check_h=checks.check_motor_height (p1.c1,p1.c2,p1.c3,p1.D)
                if check_h==-1:
                    print("Motor choca con el plato:\n")
                    "[BUG]:si el motor choca contra el plato y al mismo tiempo el punto es unreachable no se puede generar el"
                    "Script de geogebra, porque no hay angulos para armar los limbs"
                    #p1.generate_geogebra()
                P1,P2,P3=p1.calculate_P()
                name="P"+str(point)
                s1.add_2D_point(P1,name)
                point=point+1

In [4]:
pyperclip.copy(s1.print_script())

Execute[{
"P0=(4.000,8.539)"
,"P1=(4.000,8.699)"
,"P2=(4.000,8.825)"
,"P3=(4.000,8.915)"
,"P4=(3.835,10.000)"
,"P5=(3.835,10.000)"
,"P6=(4.000,11.085)"
,"P7=(4.000,11.175)"
,"P8=(4.000,11.301)"
,"P9=(4.606,7.725)"
,"P10=(4.496,12.068)"
,"P11=(4.000,8.881)"
,"P12=(4.000,9.031)"
,"P13=(4.000,9.141)"
,"P14=(3.899,10.000)"
,"P15=(3.899,10.000)"
,"P16=(4.000,10.859)"
,"P17=(4.000,10.969)"
,"P18=(4.378,8.184)"
,"P19=(4.496,7.932)"
,"P20=(4.413,11.894)"
,"P21=(4.290,11.597)"
,"P22=(4.000,9.209)"
,"P23=(4.000,9.349)"
,"P24=(3.947,10.000)"
,"P25=(3.947,10.000)"
,"P26=(4.000,10.651)"
,"P27=(4.198,8.676)"
,"P28=(4.290,8.403)"
,"P29=(4.413,8.106)"
,"P30=(4.358,11.768)"
,"P31=(4.232,11.430)"
,"P32=(4.136,11.101)"
,"P33=(4.000,9.529)"
,"P34=(4.000,9.649)"
,"P35=(4.000,10.351)"
,"P36=(4.073,9.194)"
,"P37=(4.136,8.899)"
,"P38=(4.232,8.570)"
,"P39=(4.358,8.232)"
,"P40=(4.495,8.094)"
,"P41=(4.303,8.483)"
,"P42=(4.158,8.890)"
,"P43=(4.041,10.604)"
,"P44=(4.000,9.844)"
,"P45=(4.008,9.729)"
,"P46=(4.041,9.

In [5]:
p1=platform(d=6,D=10,l1=6,l2=12,max_limb_error=0.00001,max_limb_try_count=30,max_joint_angle=23*np.pi/180)
p1.solve_platform(r_x=0*np.pi/180,r_y=20*np.pi/180,h=10) 

0

In [6]:
geogebra_script=geogebra.geogebra_script()
pyperclip.copy(geogebra_script.generate_geogebra(p1))

Execute[{
"C1=(0.000,-5.457,11.986)"
,"C2=(-5.196,3.000,8.908)"
,"C3=(5.196,3.000,8.908)"
,"Polygon(C1,C2, C3)"
,"M1=(0.000,-10.000,0.000)"
,"M2=(-8.660,5.000,0.000)"
,"M3=(8.660,5.000,0.000)"
,"AUX0=(0.000,0.000,0.000)"
,"AUX4=(0.000,0.000,1.000)"
,"Plane(M1,AUX0, AUX4)"
,"Plane(M2,AUX0, AUX4)"
,"Plane(M3,AUX0, AUX4)"
,"A1=(0.000,-4.000,0.075)"
,"Segment(M1,A1)"
,"Segment(A1,C1)"
,"A2=(-4.177,2.412,-3.034)"
,"Segment(M2,A2)"
,"Segment(A2,C2)"
,"A3=(4.177,2.412,-3.034)"
,"Segment(M3,A3)"
,"Segment(A3,C3)"
,"AUX1=(0.000,-12.000,2.500)"
,"AUX2=(5.000,-12.000,2.500)"
,"Segment(AUX1,AUX2)"
}]
